In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:

data = pd.read_csv('/Users/pierros/Documents/Courses/ECE6254/Projects/Final Project/features30sec.csv')
data = data.iloc[0:, 1:] 
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,pop.00027.wav,30.000181,0.377612,0.085683,0.157902,0.002739,2682.284141,8.052991e+05,2923.967729,190731.575470,...,1.0,2.950268e-09,1.0,5.900537e-09,1.0,0.000000e+00,1.0,0.000000e+00,1.0,pop
1,pop.00033.wav,30.000181,0.423036,0.085342,0.139896,0.002499,2835.119375,4.375327e+05,2731.752387,163886.609471,...,1.0,-1.180107e-08,1.0,-1.180107e-08,1.0,1.475134e-08,1.0,1.180107e-08,1.0,pop
2,pop.00032.wav,30.000181,0.426769,0.089141,0.097861,0.002146,4225.461049,1.637053e+06,3404.380733,105212.009597,...,1.0,1.180107e-08,1.0,0.000000e+00,1.0,5.900537e-09,1.0,-5.900537e-09,1.0,pop
3,pop.00026.wav,30.000181,0.334962,0.090551,0.195626,0.007856,2525.591959,1.009068e+06,2848.409561,276177.417023,...,1.0,-7.375672e-09,1.0,5.900537e-09,1.0,1.180107e-08,1.0,2.950268e-09,1.0,pop
4,pop.00030.wav,30.000181,0.357630,0.084706,0.203093,0.006338,3070.672741,6.090035e+05,2975.765677,125019.755279,...,1.0,-2.950268e-09,1.0,-5.900537e-09,1.0,-2.950268e-09,1.0,5.900537e-09,1.0,pop


In [3]:
data = data.drop(["filename","length"], axis=1)

In [4]:
y = data['label'] # genre variable.
X = data.loc[:, data.columns != 'label'] #select all columns but not the labels

#### NORMALIZE X ####

# Normalize so everything is on the same scale. 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')
    return round(accuracy_score(y_test, preds), 5)

Native Bayes

In [7]:
nb = GaussianNB()
score = model_assess(nb, "Naive Bayes")

Accuracy Naive Bayes : 0.45333 



Stochastic Gradient Descent

In [8]:
sgd = SGDClassifier(max_iter=500, random_state=0)
score = model_assess(sgd, "Stochastic Gradient Descent")

Accuracy Stochastic Gradient Descent : 0.37667 



KNN

In [9]:
knn_scores = []

for k in range(1,500):
    knn = KNeighborsClassifier(n_neighbors=k)
    # print(k)
    score = model_assess(knn, "KNN")
    knn_scores.append(score)

# Find index with the lowest error
df_knn = pd.DataFrame(knn_scores)
df_knn.index = np.arange(1, len(df_knn) + 1)
best_k = df_knn.idxmax()[0]
print('Best K:',best_k, 'Value:',df_knn.max()[0])


Accuracy KNN : 0.26333 

Accuracy KNN : 0.28667 

Accuracy KNN : 0.26333 

Accuracy KNN : 0.30667 

Accuracy KNN : 0.30333 

Accuracy KNN : 0.32 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.30333 

Accuracy KNN : 0.3 

Accuracy KNN : 0.30333 

Accuracy KNN : 0.31 

Accuracy KNN : 0.32 

Accuracy KNN : 0.30333 

Accuracy KNN : 0.33 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.36 

Accuracy KNN : 0.34667 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.35 

Accuracy KNN : 0.36 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.36667 

Accuracy KNN : 0.35 

Accuracy KNN : 0.35 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.35 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.34667 

Accuracy KNN : 0.34 

Accuracy KNN : 0.36333 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.33 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.32333 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.33667 

Accuracy KNN : 0.34 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.3

In [10]:
best_k

67

In [11]:
# With tuned parameter
knn = KNeighborsClassifier(n_neighbors=best_k)
score = model_assess(knn, "KNN")

Accuracy KNN : 0.40667 



Decission trees

In [12]:
tree = DecisionTreeClassifier()
score = model_assess(tree, "Decission trees")

Accuracy Decission trees : 0.39667 



Random Forest

In [14]:
rf_scores = []
for est in tqdm(range(1000,3000,100)):
    d_scores = []
    for d in range(1,1000,100):
        rforest = RandomForestClassifier(n_estimators=est, max_depth=d, random_state=0)
        score = model_assess(rforest, "Random Forest")
        d_scores.append(score)
    rf_scores.append(d_scores)

df_rf = pd.DataFrame(rf_scores)

best_rf_param = np.argwhere(rf_scores == np.max(rf_scores))[0]

print("Best n_est:",best_rf_param[0],"Best depth:",best_rf_param[1])

  0%|          | 0/20 [00:00<?, ?it/s]

Accuracy Random Forest : 0.32667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.59667 



  5%|▌         | 1/20 [00:25<08:05, 25.55s/it]

Accuracy Random Forest : 0.59667 

Accuracy Random Forest : 0.32333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 

Accuracy Random Forest : 0.59333 



 10%|█         | 2/20 [00:52<07:52, 26.25s/it]

...

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.32333 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 



 95%|█████████▌| 19/20 [14:33<01:01, 61.65s/it]

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.32667 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 



100%|██████████| 20/20 [15:34<00:00, 46.70s/it]

Accuracy Random Forest : 0.6 

Best n_est: 10 Best depth: 1


In [15]:
print(best_rf_param[0]+1, best_rf_param[1]+1)


11 2


In [16]:
# With tuned parameters
rforest = RandomForestClassifier(n_estimators=best_rf_param[0]+1, max_depth=best_rf_param[1]+1, random_state=0)
score = model_assess(rforest, "Random Forest")

Accuracy Random Forest : 0.35333 



Support Vector Machine

In [17]:
svm = SVC(decision_function_shape="ovo")
score = model_assess(svm, "Support Vector Machine")
svm = SVC(decision_function_shape="ovr")
score = model_assess(svm, "Support Vector Machine")


Accuracy Support Vector Machine : 0.45 

Accuracy Support Vector Machine : 0.45 



Logistic Regression

In [18]:
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
score = model_assess(lg, "Logistic Regression")

Accuracy Logistic Regression : 0.46 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Neural Nets

In [19]:
best_score_mlp = 0
for a in np.linspace(0.00001,0.0001,20):
    nn = MLPClassifier(solver='lbfgs', alpha=a, hidden_layer_sizes=(5000, 10), random_state=1)
    score = model_assess(nn, "Neural Nets")
    if score > best_score_mlp:
        best_score_mlp = score
        best_alpha = a

/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy Neural Nets : 0.46667 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy Neural Nets : 0.47333 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy Neural Nets : 0.45667 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy Neural Nets : 0.48 

Accuracy Neural Nets : 0.44 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [20]:
print(best_alpha)

7.157894736842105e-05


In [21]:
nn = MLPClassifier(solver='lbfgs', alpha=best_alpha, hidden_layer_sizes=(5000, 10), random_state=1)
score = model_assess(nn, "Neural Nets")

Accuracy Neural Nets : 0.5 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [22]:
# Transform Data for the next classifiers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train2 = le.fit_transform(y_train)
y_test2 =le.fit_transform(y_test)

Cross Gradient Booster

In [23]:
best_cgb = [1,0.01]
best_score_cgb = 0
for est in tqdm(range(1000,2000,100)):
    for lr in np.linspace(0.1,0.7,7):
        xgb = XGBClassifier(n_estimators=est, learning_rate=lr)
        # model_assess(xgb, "Cross Gradient Booster")
        xgb.fit(X_train, y_train2)
        preds = xgb.predict(X_test)
        #print(confusion_matrix(y_test, preds))
        print('Accuracy', "Cross Gradient Booster", ':', round(accuracy_score(y_test2, preds), 5), '\n')
        if round(accuracy_score(y_test2, preds), 5) > best_score_cgb:
            best_score_cgb = round(accuracy_score(y_test2, preds), 5)
            best_cgb = [est, lr]

  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy Cross Gradient Booster : 0.63333 

Accuracy Cross Gradient Booster : 0.62333 

Accuracy Cross Gradient Booster : 0.62 

Accuracy Cross Gradient Booster : 0.61 

Accuracy Cross Gradient Booster : 0.62 

Accuracy Cross Gradient Booster : 0.59333 



 10%|█         | 1/10 [08:37<1:17:34, 517.16s/it]

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.64 

Accuracy Cross Gradient Booster : 0.62 

Accuracy Cross Gradient Booster : 0.61667 

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.61667 

Accuracy Cross Gradient Booster : 0.59667 



 20%|██        | 2/10 [16:58<1:07:44, 508.11s/it]

...

Accuracy Cross Gradient Booster : 0.61667 

Accuracy Cross Gradient Booster : 0.65 

Accuracy Cross Gradient Booster : 0.62 

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.62333 

Accuracy Cross Gradient Booster : 0.6 



 90%|█████████ | 9/10 [1:33:26<11:20, 680.92s/it]

Accuracy Cross Gradient Booster : 0.61667 

Accuracy Cross Gradient Booster : 0.65 

Accuracy Cross Gradient Booster : 0.62 

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.61333 

Accuracy Cross Gradient Booster : 0.62333 

Accuracy Cross Gradient Booster : 0.6 



100%|██████████| 10/10 [1:43:50<00:00, 623.00s/it]

Accuracy Cross Gradient Booster : 0.61667 



In [24]:
best_cgb

[1400, 0.1]

In [25]:
xgb = XGBClassifier(n_estimators=best_cgb[0], learning_rate=best_cgb[1])
xgb.fit(X_train, y_train2)
preds = xgb.predict(X_test)
print('Accuracy', "Cross Gradient Booster", ':', round(accuracy_score(y_test2, preds), 5), '\n')

Accuracy Cross Gradient Booster : 0.65333 



Cross Gradient Booster (Random Forest)

In [26]:
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
# model_assess(xgbrf, "Cross Gradient Booster (Random Forest)")
xgbrf.fit(X_train, y_train2)
preds = xgbrf.predict(X_test)
print('Accuracy', "Cross Gradient Booster (Random Forest)", ':', round(accuracy_score(y_test2, preds), 5), '\n')

Accuracy Cross Gradient Booster (Random Forest) : 0.57667 



In [27]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(estimator=xgb, random_state=1)
perm.fit(X_test, y_test2)

eli5.show_weights(estimator=perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.1773 ± 0.0481,chroma_stft_mean
0.1187 ± 0.0269,spectral_bandwidth_mean
0.0840 ± 0.0208,chroma_stft_var
0.0653 ± 0.0191,perceptr_mean
0.0633 ± 0.0263,harmony_var
0.0600 ± 0.0316,rms_var
0.0600 ± 0.0256,tempo
0.0580 ± 0.0259,rolloff_var
0.0473 ± 0.0190,spectral_centroid_var
0.0460 ± 0.0236,rolloff_mean
